# 3 Language Modeling

In [35]:
import nltk
import math
import random

In [22]:
tokens = nltk.word_tokenize(open('training_data.txt', encoding='utf-8').read())

In [23]:
def set_UNK(words):
  count = {}
  for word in words:
    try:
      count[word] += 1
    except:
      count[word] = 1
  for i in range(len(words)):
    if count[words[i]]<=3:
      words[i] = 'UNK'
  return words

In [90]:
unigram = {}
bigram = {}
trigram = {}
def train(tokens):
    tokens = set_UNK(tokens)
    unigram = {}
    for token in tokens:
        try:
            unigram[token] += 1
        except:
            unigram[token] = 1

    bigram = {}
    for i in range(1,len(tokens)):
        try:
            bigram[(tokens[i-1],tokens[i])] += 1
        except:
            bigram[(tokens[i-1],tokens[i])] = 1

    trigram = {}
    for i in range(2,len(tokens)):
        try:
            trigram[(tokens[i-2],tokens[i-1],tokens[i])] += 1
        except:
            trigram[(tokens[i-2],tokens[i-1],tokens[i])] = 1
    return unigram, bigram, trigram
unigram, bigram, trigram = train(tokens)

In [7]:
# write to file
import json
with open('unigram.txt','w') as f:
    f.write(str(unigram))
with open('trigram.txt','w') as f:
    f.write(str(trigram))
with open('bigram.txt','w') as f:
    f.write(str(bigram))

# 3.1

In [26]:
def uni(key):
    if key in unigram:
        return unigram[key]
    else:
        return unigram['UNK']

def bi(key):
    if key in bigram:
        return bigram[key]
    else:
        return 0

def tri(key):
    if key in trigram:
        return trigram[key]
    else:
        return 0


$$
P(w_n)=\frac{N(w_n)}{N(words)}\\
P(w_n|w_{n-1})=\frac{N(w_{n-1},w_n)}{N(w_{n-1})}\\
P(w_n|w_{n-2},w_{n-1})=\frac{N(w_{n-2},w_{n-1},w_n)}{N(w_{n-2},w_{n-1})}
$$


In [68]:
def cal_pp_linear(words, lamda):
    N = len(words)
    n_uni = len(words)
    log_P = 0
    for i in range(2, N):
        P_uni = uni(words[i])/n_uni
        P_bi = bi((words[i-1],words[i]))/uni(words[i-1])
        try:
            P_tri = tri((words[i-2],words[i-1],words[i]))/bi((words[i-2],words[i-1]))
        except:
            P_tri = 0
        P_i = P_uni*lamda[0] + P_bi*lamda[1] + P_tri*lamda[2]
        
        try:
            log_P += math.log(P_i)
        except:
            print(words, str(lamda))
    return math.exp(-log_P/N)

In [96]:
held_out = tokens[:math.floor(0.2*len(tokens))]
unigram, bigram, trigram = train(tokens[math.floor(0.2*len(tokens)):])
##

lamdas = [[0.8, 0.1, 0.1],
      [0.6, 0.2, 0.2],
      [0.4, 0.3, 0.3],
      [0.1, 0.8, 0.1],
      [0.2, 0.6, 0.2],
      [0.3, 0.4, 0.3],
      [0.1, 0.1, 0.8],
      [0.2, 0.2, 0.6],
      [0.3, 0.3, 0.4],
      ]

# find lambda
def find_lambda(words, lamdas):
    min_perplexity = float('inf')
    min_lamda = []
    for lamda in lamdas:
        perplexity = cal_pp_linear(words, lamda)
        print('lambda: %s perplexity: %f'%(str(lamda), perplexity))
        if perplexity < min_perplexity:
            min_perplexity = perplexity
            min_lamda = lamda
    return min_lamda

best_lambda = find_lambda(held_out, lamdas)

lambda: [0.8, 0.1, 0.1] perplexity: 85.699660
lambda: [0.6, 0.2, 0.2] perplexity: 76.248407
lambda: [0.4, 0.3, 0.3] perplexity: 74.142995
lambda: [0.1, 0.8, 0.1] perplexity: 89.932246
lambda: [0.2, 0.6, 0.2] perplexity: 79.219987
lambda: [0.3, 0.4, 0.3] perplexity: 75.163013
lambda: [0.1, 0.1, 0.8] perplexity: 107.639573
lambda: [0.2, 0.2, 0.6] perplexity: 83.932997
lambda: [0.3, 0.3, 0.4] perplexity: 75.808752


In [97]:
test_words = nltk.word_tokenize(open('testing_data.txt', encoding='utf-8').read())
test_words = set_UNK(test_words)

unigram, bigram, trigram = train(tokens)

In [98]:
cal_pp_linear(test_words,best_lambda)

66.55379562841097

# 3.3

In [19]:
#bigram
def cal_pp_add(words, lamda):
  N = len(words)
  V = len(words)
  log_P = 0
  for i in range(1, V):
    P = (bi((words[i-1],words[i])) + lamda)/(uni(words[i-1]) + lamda*V)
    log_P += math.log(P)
  return math.exp(-log_P/N)

In [20]:
print(cal_pp_add(test_words, 0.1))
print(cal_pp_add(test_words, 0.2))
print(cal_pp_add(test_words, 0.3))

1533.108936679848
2512.675687167015
3417.2315494894638
